In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from scipy import stats
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import PartialDependenceDisplay, partial_dependence
from sklearn.svm import SVR
from sklearn.cluster import DBSCAN
import time
import matplotlib.pylab as pylab
# params = {'legend.fontsize':12,
#           'axes.labelsize':16,
#          'axes.titlesize':16,
#          'xtick.labelsize':12,
#          'ytick.labelsize':12}
# pylab.rcParams.update(params)

In [4]:
def convert(df):
    for col in df.columns:
        if df[col].notna().all():
            if col != 'Wind_Speed':
                df[col] = df[col].astype(int)
            else:
                df[col] = round(df[col], 1)

    return df

In [5]:
def custom_round(x, base):
    
    return int(base * round(float(x)/base))

In [ ]:
def remove_outliers(df):
    feature_1 = 'Wind_Speed'
    feature_2 = 'Prod_Pwr'
    X = df.loc[:'2016'][[feature_1, feature_2]].values
    db = DBSCAN(eps=1.4, min_samples=10).fit(X)
    labels = db.labels_
    
    return labels

In [1]:
# Process and clean the data. Requires csv files to be located in a folder called "CSV" in the directory of the notebook.
def process_data():     
    # Load the data
    df_signals_2016 = pd.read_csv('CSV\wind-farm-signals-2016.csv', sep=';')
    df_signals_2017 = pd.read_csv('CSV\wind-farm-signals-2017_bruk.csv', sep=';')
    df_metmast_2016 = pd.read_csv('CSV\wind-farm-metmast-2016.csv', sep=';')
    df_metmast_2017 = pd.read_csv('CSV\wind-farm-metmast-2017.csv', sep=';')
    
    # Merge dataframes
    frames1 = [df_signals_2016, df_signals_2017]
    df1 = pd.concat(frames1, ignore_index=True)
    # Convert to datetime and remove time zone
    df1['Timestamp'] = pd.to_datetime(df1['Timestamp'], dayfirst=True).dt.tz_localize(None)

    frames2 = [df_metmast_2016, df_metmast_2017]
    df2 = pd.concat(frames2, ignore_index=True)
    df2['Timestamp'] = pd.to_datetime(df2['Timestamp'], dayfirst=True).dt.tz_localize(None)

    # Join dataframes on index 'Timestamp'
    df = df1.set_index('Timestamp').join(df2.set_index('Timestamp'))

    # Select turbine
    turbine = 'T07'
    df = df.loc[df['Turbine_ID'] == turbine].reset_index()

    # Extract only relevant columns
    df = df[['Timestamp', 'Turbine_ID', 'Gen_Bear_Temp_Avg', 'Gen_Bear2_Temp_Avg',
             'Gen_RPM_Avg', 'Nac_Temp_Avg', 'Amb_WindSpeed_Avg', 'Avg_Humidity', 
             'Gen_Phase1_Temp_Avg', 'Gen_Phase2_Temp_Avg', 'Gen_Phase3_Temp_Avg',
             'Amb_Temp_Avg', 'Grd_Prod_Pwr_Avg',
             'Amb_WindDir_Abs_Avg',
             'Rtr_RPM_Avg'
            ]].copy()

    # Rename columns
    df = df.rename(columns={'Timestamp': 'Date',
                            'Gen_Bear_Temp_Avg': 'Gen_Bear_Temp',
                            'Gen_RPM_Avg': 'Gen_RPM',
                            'Gen_Bear2_Temp_Avg': 'Gen_Bear2_Temp',
                            'Nac_Temp_Avg': 'Nac_Temp', 
                            'Amb_WindSpeed_Avg': 'Wind_Speed', 
                            'Avg_Humidity': 'Humidity',
                            'Gen_Phase1_Temp_Avg': 'Gen_Phase1_Temp', 
                            'Gen_Phase2_Temp_Avg': 'Gen_Phase2_Temp',
                            'Gen_Phase3_Temp_Avg': 'Gen_Phase3_Temp',
                            'Grd_Prod_Pwr_Avg': 'Prod_Pwr',
                            'Amb_WindDir_Abs_Avg': 'Wind_Dir',
                            'Amb_Temp_Avg': 'Amb_Temp',
                            'Rtr_RPM_Avg': 'Rtr_RPM',
                            'Grd_Prod_Pwr_Avg': 'Prod_Pwr'})

    df['Gen_Phase_Temp'] = df[['Gen_Phase1_Temp', 'Gen_Phase2_Temp', 'Gen_Phase3_Temp']].mean(axis=1)
    df = df.drop(columns=['Gen_Phase1_Temp', 'Gen_Phase2_Temp', 'Gen_Phase3_Temp'])

    # Fill the missing Gen_Bear_Temp nan value with the the mean of the values diorectly next to it
    df['Gen_Bear_Temp'] = df['Gen_Bear_Temp'].fillna(48)
    df['Humidity'] = df['Humidity'].interpolate(method='linear').round()
#     df = df.loc[df['Gen_Bear_Temp'] < 100]

    # Combine duplicates by their mean
    df = df.groupby(df['Date']).mean(numeric_only=True).reset_index()
    df = df.set_index('Date')

    df = df.round(1)

    df = df.query('~(Gen_Bear_Temp > 100 & index < 2017)')
    df = df.query('~(Prod_Pwr < 0 & Wind_Speed >= 4 & index < 2017)')
    
    labels = remove_outliers(df)
    labels = pd.Series(labels, df.loc[:'2016'].index, name='Label')
    df = pd.concat([df, labels], axis=1) 
    df['Label'] = df['Label'].fillna(0)
    df['Label'] = df['Label'].astype('int')
    df = df.query('Label == 0')
        
    return df

In [89]:
# Train models
def train_models(X_train, y_train):

#     start = time.time()
#     LR = LinearRegression(n_jobs=-1)
#     LR.fit(X_train, y_train)
#     end = time.time()
#     print(end - start)
    
#     start = time.time()
#     RF = RandomForestRegressor(random_state=42)
#     RF.fit(X_train, y_train)
#     end = time.time()
#     print(end - start)
    
#     start = time.time()
#     SVM = make_pipeline(StandardScaler(), SVR())
#     SVM.fit(X_train, y_train)
#     end = time.time()
#     print(end - start)
    
    start = time.time()
    XGB = xgb.XGBRegressor(
        booster='gbtree',    
        n_estimators=1000,
        objective='reg:squarederror',
        max_depth=4,
        learning_rate=0.05,
        colsample_bytree =0.6,
        colsample_bylevel=0.6,
        verbosity=0,
        n_jobs=-1,
        random_state=42
    )
    XGB.fit(X_train, y_train)
    end = time.time()
    print(end - start)  

    models = [
#          LR, RF, SVM,
              XGB]

    return models

In [8]:
# Cross validate models on folds
def CV(df, models, features, target):
    cv = df.loc[df.index < '2017'].copy()
    tss = TimeSeriesSplit(n_splits=4, test_size=6*24*30, gap=6*24)
    
    for model in models:
        
        fold = 0
        preds = []
        scores = []
        
        for train_idx, val_idx in tss.split(cv):

            train = cv.iloc[train_idx]
            test = cv.iloc[val_idx]
            
            X_train = train[features]
            y_train = train[target]
            
            X_test = test[features]
            y_test = test[target]
            
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            preds.append(y_pred)
            score = np.sqrt(mean_squared_error(y_test, y_pred))
            scores.append(score)
            
        print(str(model)[:12])
        print(f'Score across folds {np.mean(scores):0.2f}')
        scores = [ '%.2f' % score for score in scores ]
        print(f'Fold scores:{scores} \n')

In [ ]:
# Function for creating predictions
def predict(X_test, df, models):
    test_index = df.loc[df.index >= '2017'].index
    test = pd.DataFrame(index=test_index)
    
    for model in models:
        col = str(model)[:3]
        start = time.time()
        test[col] = model.predict(X_test)
        end = time.time()
        print(end - start)
    
    df = df.merge(test, how='left', left_index=True, right_index=True).copy()
#     df = convert(df)
    df = df.astype('float')
    df = df.round(1)

    return df

In [81]:
# Function for calculating metrics of ML models
def metric(df, models, target):
    ind = [str(model)[:3] for model in models]
    cols = ['MAE', 'MAPE', 'MSE', 'RMSE', 'R2']
    metrics = pd.DataFrame(index=ind, columns=cols)
    df = df[df['XGB'].notna()]

    for model in models:
        col = str(model)[:3]
        MSE = mean_squared_error(df[target], df[col])
        RMSE = np.sqrt(mean_squared_error(df[target], df[col]))
        MAE = mean_absolute_error(df[target], df[col])
        MAPE = mean_absolute_percentage_error(df[target], df[col])
        R2 = r2_score(df[target], df[col])
        
        metrics.loc[col] = MAE, MAPE, MSE, RMSE, R2
        metrics = metrics.astype('float').round(4)
        
    return metrics

In [88]:
# Function to select params to tune using GridSearch
def param_selection(model):
    model = str(model)
    if model.startswith('XGB'):
        param_grid = {
            'max_depth': [3, 4, 5, 6, 7, 8, 9],
            'n_estimators': range(200, 1200, 200),
            'learning_rate': [0.1, 0.05, 0.01],
        }
        
    elif model.startswith('Hist'):
        param_grid = {
            'max_depth': [2, 3, 4, 5],
            'learning_rate': [0.1, 0.01, 0.001],
            'max_leaf_nodes': [3, 10, 30]}
        
    elif model.startswith('LGB'):
        param_grid = {
            'max_depth': [2, 3, 4, 5],
            'learning_rate': [0.1, 0.01, 0.001],
            'n_estimators': range(500, 1000, 100),
            'num_leaves': range(50, 100, 10),
            'min_data_in_leaf': range(100, 1100, 200)}
        
    elif model.startswith('Ada'):
        param_grid = {
            'loss': ['linear', 'square', 'exponential'],            
            'learning_rate': [0.1, 0.01],
#             'n_estimators': range(500, 1100, 100)}
            'n_estimators': [50, 100, 200, 400]}
        
    elif model.startswith('RandomForest'):
        param_grid = {
            'max_depth': [3, 5, 7],
            'n_estimators': [100, 200, 300],
            'min_samples_leaf': [1, 2, 3]}  
        
    return param_grid

In [ ]:
def best_params(X_train, y_train, search):
    search.fit(X_train, y_train)
    
    grid = pd.DataFrame(search.cv_results_)

    print(f"The best parameters are {search.best_params_} with a score of {round(search.best_score_, 2)}")
    
    return grid

In [ ]:
# Calculate error and show timestamps of largest errors
def errors(df, models, target):
    for model in models:
        col = str(model)[:3]
        df[f'error_{col}'] = np.abs(df[df[col].notna()][target] - df[df[col].notna()][col])

    return df